<a href="https://colab.research.google.com/github/ilija-ra/ElectricityConsumptionForecast.BE/blob/main/ElectricityConsumptionForecast_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install genetic
# !pip install deap
# !pip install scikeras

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasRegressor

In [51]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = tf.squeeze(y_true), tf.squeeze(y_pred)
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

In [52]:
# Load your dataset
data = pd.read_csv("final_preprocessing.csv")
data.drop('datetime', axis = 1, inplace= True)
data.drop('Unnamed: 0', axis = 1, inplace= True)
data['Load'].interpolate(inplace = True)

In [53]:
# Select features and target variable
features = data.drop("Load", axis=1)
target = data["Load"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [54]:
# Build the neural network model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)  # For regression, no activation function in the output layer
])

# Compile the model
model.compile(optimizer='SGD', loss='mean_absolute_percentage_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=10, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss}%")
# result: 5.654926776885986%

Epoch 1/10
2056/2056 [==============================] - 5s 2ms/step - loss: 99.9229 - val_loss: 99.6315
Epoch 2/10
2056/2056 [==============================] - 4s 2ms/step - loss: 34.4414 - val_loss: 7.7392
Epoch 3/10
2056/2056 [==============================] - 6s 3ms/step - loss: 7.4966 - val_loss: 7.5316
Epoch 4/10
2056/2056 [==============================] - 5s 2ms/step - loss: 7.1252 - val_loss: 7.1020
Epoch 5/10
2056/2056 [==============================] - 4s 2ms/step - loss: 6.8049 - val_loss: 6.6375
Epoch 6/10
2056/2056 [==============================] - 6s 3ms/step - loss: 6.6023 - val_loss: 8.2170
Epoch 7/10
2056/2056 [==============================] - 4s 2ms/step - loss: 6.4840 - val_loss: 6.5657
Epoch 8/10
2056/2056 [==============================] - 5s 3ms/step - loss: 6.4037 - val_loss: 7.3693
Epoch 9/10
2056/2056 [==============================] - 8s 4ms/step - loss: 6.1503 - val_loss: 5.6950
Epoch 10/10
201/201 [==============================] - 0s 2ms/step - loss: 5.87

In [55]:
# keras.models.load_model("primary_neural_network.keras")

**HYPERPARAMETERS TUNING**

In [96]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
import numpy as np

In [97]:
# Define your Keras model as a function
def create_model(optimizer='adam', epochs=150, batch_size=10, validation_split=0.2):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=optimizer, loss='mean_absolute_percentage_error')

    return model

In [98]:
data = pd.read_csv("final_preprocessing.csv")
data.drop('datetime', axis = 1, inplace= True)
data.drop('Unnamed: 0', axis = 1, inplace= True)
data['Load'] = data['Load'].astype(float)
data['Load'].interpolate(inplace = True)

In [99]:
y_train = data['Load']
X_train = data.drop('Load', axis=1)

In [100]:
# # Assuming data is loaded and processed as in your original code
# # Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [84]:
# Create a KerasRegressor with your model function
keras_regressor = KerasRegressor(build_fn=create_model, verbose=0)

In [101]:
# Define the hyperparameter grid
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'epochs': [50, 100, 150],
    'batch_size': [16, 32, 64],
    'validation_split': [0.1, 0.2, 0.3]
}

In [103]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [104]:
# Define the scoring metric (mean absolute error in this case)
mae_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [105]:
# Create the GridSearchCV object
grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, scoring=mae_scorer, cv=3)

In [ ]:
# Fit the grid search to the data
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:", grid_result.best_params_)

In [ ]:
# Get the best model
best_model = grid_result.best_estimator_.model